# Train model on common_3d_test_models
The idea is to train a DGCNN on the objects of the common_3d_test_models.
First, we preprocess all models to create patch files and then we train the network on the files.

In [1]:
import numpy as np
import scipy.stats as stats
import igl

from PatchGeneration.Modules.Mesh import *
from PatchGeneration.Modules.PatchCollector import *

In [7]:
obj_dir = "./common-3d-test-models-master"
corrupted_files = np.array(["beast.obj", "suzanne.obj"])
filenames = np.array(os.listdir(obj_dir))
filenames = np.setdiff1d(filenames, corrupted_files)
filenames = filenames[np.char.endswith(filenames, ".obj")]
meshes = np.array([Mesh.readFile(f"{obj_dir}/{filename}") for filename in filenames])

Filter out outliers with too little or too much vertices.

In [8]:
sizes = np.array([len(mesh.getVertices()) for mesh in meshes])
iqr = stats.iqr(sizes)
median = np.median(sizes)
thresholds = np.logical_or(sizes > median + 2 * iqr, sizes < median - 2 * iqr)
obj_set = meshes[np.logical_not(thresholds)]
filenames = filenames[np.logical_not(thresholds)]
print(f"{thresholds.sum()} meshes have been filtered out, because they have too little or too many vertices")

2 meshes have been filtered out, because they have too little or too many vertices


For every selected model, create noise and save it as .obj files.

In [11]:
os.makedirs(f"{obj_dir}/Generated_Noise")
for idx, mesh in enumerate(obj_set):
    for noise_level in range(3):
        noisy_mesh = mesh.copy()
        noisy_mesh.applyGaussianNoise((1 + noise_level) * 0.1)
        save_path = f"{obj_dir}/Generated_Noise/{filenames[idx][:-4]}_{noise_level + 1}.obj"
        igl.write_obj(save_path, noisy_mesh.getVertices(), noisy_mesh.f)
        print(f"Written {save_path}")

Written ./common-3d-test-models-master/Generated_Noise/alligator_1.obj
Written ./common-3d-test-models-master/Generated_Noise/alligator_2.obj
Written ./common-3d-test-models-master/Generated_Noise/alligator_3.obj
Written ./common-3d-test-models-master/Generated_Noise/armadillo_1.obj
Written ./common-3d-test-models-master/Generated_Noise/armadillo_2.obj
Written ./common-3d-test-models-master/Generated_Noise/armadillo_3.obj
Written ./common-3d-test-models-master/Generated_Noise/beetle-alt_1.obj
Written ./common-3d-test-models-master/Generated_Noise/beetle-alt_2.obj
Written ./common-3d-test-models-master/Generated_Noise/beetle-alt_3.obj
Written ./common-3d-test-models-master/Generated_Noise/beetle_1.obj
Written ./common-3d-test-models-master/Generated_Noise/beetle_2.obj
Written ./common-3d-test-models-master/Generated_Noise/beetle_3.obj
Written ./common-3d-test-models-master/Generated_Noise/bimba_1.obj
Written ./common-3d-test-models-master/Generated_Noise/bimba_2.obj
Written ./common-3d-

Calculate patches from object files in the Generated_Noise dictionary.

In [14]:
input_dir = obj_dir + "/Generated_Noise"
input_files = os.listdir(input_dir)
meshes = np.array([Mesh.readFile(f"{input_dir}/{filename}") for filename in input_files])

os.makedirs(f"{obj_dir}/Patches")

for idx, mesh in enumerate(meshes):
    patch_collector = PatchCollector(noisy_mesh)
    save_dir = f"{obj_dir}/Patches/{input_files[idx][:-4]}"
    print(save_dir)
    # patch_collector.collectAndSavePatches(save_dir)

./common-3d-test-models-master/Patches/alligator_1
./common-3d-test-models-master/Patches/alligator_2
./common-3d-test-models-master/Patches/alligator_3
./common-3d-test-models-master/Patches/armadillo_1
./common-3d-test-models-master/Patches/armadillo_2
./common-3d-test-models-master/Patches/armadillo_3
./common-3d-test-models-master/Patches/beetle-alt_1
./common-3d-test-models-master/Patches/beetle-alt_2
./common-3d-test-models-master/Patches/beetle-alt_3
./common-3d-test-models-master/Patches/beetle_1
./common-3d-test-models-master/Patches/beetle_2
./common-3d-test-models-master/Patches/beetle_3
./common-3d-test-models-master/Patches/bimba_1
./common-3d-test-models-master/Patches/bimba_2
./common-3d-test-models-master/Patches/bimba_3
./common-3d-test-models-master/Patches/cheburashka_1
./common-3d-test-models-master/Patches/cheburashka_2
./common-3d-test-models-master/Patches/cheburashka_3
./common-3d-test-models-master/Patches/cow_1
./common-3d-test-models-master/Patches/cow_2
./co